# Homework #2 | Big Data Management & Analysis
### Nina Nurrahmawati
nn1221@nyu.edu

In [1]:
# test spark
sc

<SparkContext master=local[*] appName=PySparkShell>

# Instruction
Your task is to compute the number of restaurants per each cuisine using the provided nyc_restaurant_inspections.csv data set. The CSV file is available for download under the Data Sets page on NYU Classes. The cuisine type can be extracted from the "CUISINE DESCRIPTION" column. Note that, a single restaurant may be inspected multiple times. You must use the "CAMIS" column to keep only unique restaurants while computing the number of restaurants per cuisine.

You MUST turn in a Spark notebook, AND a Spark script that works on the NYU Dumbo cluster. For your script, the input and out file path should be specified through the command line. Basically, I should be able to run your script as:

spark-submit YOUR_SCRIPT.py nyc_restaurant_inspections.csv output_folder

and it will produce an output on HDFS under output_folder.

Using "hadoop fs -getmerge" or through the notebook, I should get the expected output similar to the below:

# Code

In [2]:
# read the data
filename = 'nyc_restaurant_inspections.csv'
rdd = sc.textFile(filename, use_unicode=True).cache()

In [3]:
# check column index
list(enumerate(rdd.first().split(',')))

[(0, 'CAMIS'),
 (1, 'DBA'),
 (2, 'BORO'),
 (3, 'BUILDING'),
 (4, 'STREET'),
 (5, 'ZIPCODE'),
 (6, 'PHONE'),
 (7, 'CUISINE DESCRIPTION'),
 (8, 'INSPECTION DATE'),
 (9, 'ACTION'),
 (10, 'VIOLATION CODE'),
 (11, 'VIOLATION DESCRIPTION'),
 (12, 'CRITICAL FLAG'),
 (13, 'SCORE'),
 (14, 'GRADE'),
 (15, 'GRADE DATE'),
 (16, 'RECORD DATE'),
 (17, 'INSPECTION TYPE')]

In [4]:
# filter (cuisine type, camis id)
def extractCuisine(partId, list_of_records):
    if partId==0: 
        next(list_of_records) # skipping the first line
    import csv
    reader = csv.reader(list_of_records)
    for row in reader:
        (camis, cuisine) = (row[0], row[7])
        yield (camis, cuisine)
# take unique value, result => (camis_id, cuisine_type)
cuisine = rdd.mapPartitionsWithIndex(extractCuisine).distinct()
# count the cuisine
counts = cuisine.map(lambda x: (x[1], 1)) \
        .reduceByKey(lambda x,y: x+y) \
        .top(25, key=lambda x: x[1])

In [5]:
counts

[('American', 6002),
 ('Chinese', 2399),
 ('CafÃ©/Coffee/Tea', 1629),
 ('Other', 1296),
 ('Pizza', 1186),
 ('Italian', 1016),
 ('Mexican', 877),
 ('Japanese', 859),
 ('Latin (Cuban, Dominican, Puerto Rican, South & Central American)', 840),
 ('Bakery', 733),
 ('Caribbean', 671),
 ('Spanish', 644),
 ('Donuts', 537),
 ('Pizza/Italian', 483),
 ('Chicken', 456),
 ('Sandwiches', 406),
 ('Juice, Smoothies, Fruit Salads', 382),
 ('Hamburgers', 378),
 ('Asian', 371),
 ('Ice Cream, Gelato, Yogurt, Ices', 339),
 ('Indian', 332),
 ('Jewish/Kosher', 327),
 ('French', 319),
 ('Delicatessen', 294),
 ('Thai', 286)]

In [8]:
type(rdd.mapPartitionsWithIndex(extractCuisine).distinct().map(lambda x: (x[1], 1)) \
    .reduceByKey(lambda x,y: x+y))
#     .top(25, key=lambda x: x[1])

pyspark.rdd.PipelinedRDD